In [ ]:
workspace()

In [ ]:
function count_heads(n)
    c::Int = 0
    for i = 1:n
        c += rand(Bool)
    end
    c
end

In [ ]:
using ProgressMeter
function tt(y; p=nothing)
    if p != nothing
        next!(p)
    end
    sleep(1)
    return y^3
end
n = 10^5
dt = .001
p1 = Progress(n, dt)
p2 = Progress(n, dt)
[ tt(y) for y=linspace(0, 5, n)]
a = @spawn [ tt(y, p=p2) for y=linspace(0, 5, n)]
fetch(a)


In [2]:
length(linspace(0, 5, 5))

5

In [ ]:
tic()
a = @spawn count_heads(100000000)
toc()

In [ ]:
tic()
b = fetch(a)
toc()

b

In [ ]:
push!(LOAD_PATH, "/home/artur/BondPricing/Julia")
include("analytic_functions.jl")

In [ ]:

# Pkg.add("analytic_functions")
# using analytic_functions

# workspace()
# workspace()
reload("analytic_functions")
using analytic_functions
# const af = analytic_functions
 

In [ ]:
pardict = Dict("vb" => 60, 
               "vbh" => 62, 
                "m" => 1.,
                "c" => 6.39,
                "p" => 61.68,
                "sigmal" => .25,
                "sigmah" => .25,
                "r" => .08,
                "gross_delta" => .02,
                "iota" => .0,
                "xi" => 1.,
                "kappa" => .015,
                "alpha" => .6,
                "pi" => .27,
                "lambda" => .3)

In [ ]:
struct Overarching(V0, m, c, p, 
                           alpha, pi, 
                           r, gross_Delta, iota, 
                           xi, k, 
                           lambda, sigmal, sigmah)
    m::Float64
    
end
    

In [ ]:
pardict.a

In [ ]:
?afun.zhi_vb()

In [ ]:
function fun_wrapper(V0, vb, dict)
    m = dict["m"]
    c = dict["c"]
    p = dict["p"]
    
    r = dict["r"]
    gross_delta = dict["gross_delta"]
    iota = dict["iota"]
    
    xi = dict["xi"]
    k = dict["kappa"]
    
    alpha = dict["alpha"]
    pi = dict["pi"]
    
    _lambda = dict["lambda"]
    
    sigmal = dict["sigmal"]
    sigmah = dict["sigmah"]
    
    sigma = dict["sigmal"]
    
#     vb = afun.zhi_vb(m, c, p, sigma, r, gross_delta, iota, xi, k, alpha, pi)
    
    vt = log(V0/vb) 
    
    # afun.zhi_bond_price(vb, v, ttm, m, c, p, sigma, r, gross_delta, xi, k, alpha, pi)
    
    vbh = vb
    println(string("VBl: ", vb))
#     return afun.bond_pr_main(vt, ttm, vb, vbh,
#                           m, c, p,
#                           r, gross_delta, iota,
#                           xi, k, alpha, pi,
#                           _lambda, sigmal, sigmah)
    ttm_grid_size = 20
    return afun.get_debt_price(vt, vb, vbh, ttm_grid_size, 
                                m, c, p, 
                                r, gross_delta, iota, 
                                xi, k, 
                                alpha, pi, 
                                _lambda, sigmal, sigmah)
    
end

In [ ]:
tic()
pardict["p"] = 58.80649063906391
debtpr = fun_wrapper(100, 64.96307165308276, pardict)
toc()

In [3]:
LOAD_PATH

2-element Array{Any,1}:
 "/usr/local/julia/0.6.0/local/share/julia/site/v0.6"
 "/usr/local/julia/0.6.0/share/julia/site/v0.6"      

In [ ]:
function bond_surf_wrapper(V0, pmin, pmax, pN, vN,
                            vbmin, vbmax, vbN, dict)
    m = dict["m"]
    c = dict["c"]
    p = dict["p"]
    
    r = dict["r"]
    gross_delta = dict["gross_delta"]
    iota = dict["iota"]
    
    xi = dict["xi"]
    k = dict["kappa"]
    
    alpha = dict["alpha"]
    pi = dict["pi"]
    
    _lambda = dict["lambda"]
    
    sigmal = dict["sigmal"]
    sigmah = dict["sigmah"]
    
    sigma = dict["sigmal"]
    
    
#     pgrid = linspace(50, 70, 5)
#     vgrid = linspace(0, log(800/50), 125)
#     vbgrid = linspace(50, 65, 5)
    
    
#     bondVmax = afun.get_bond_vmax(V0, m, c, p, sigmah, r,
#                             gross_delta, iota, xi, k,
#                             alpha, pi)
    
    
    
#     cube_future = @spawn [afun.bond_pr_main(v, m, vb, vb, m, c, p,
#                               r, gross_delta, iota,xi, k, alpha, pi, _lambda,
#                               sigmal, sigmah, log(bondVmax/vb)) for p=pgrid, v=vgrid, vb=vbgrid]
#     return fetch(cube_future)
    cube = afun.pvvb_bond_surf(V0, pmin, pmax, pN, vN,
                            vbmin, vbmax, vbN,
                            m, c, p,
                            r, gross_delta, iota,
                            xi, k,
                            alpha, pi,
                            _lambda, sigmal, sigmah)
end

In [ ]:
tic()
cube = bond_surf_wrapper(100, 50, 70, 5, 100,
                          50, 70, 5, pardict)
toc()

In [ ]:
ls

In [ ]:
cube

In [ ]:
pgrid = linspace(50, 70, 5)
vgrid = linspace(0, log(800/50), 125)
vbgrid = linspace(50, 65, 5)
itpc = interpolate(cube, BSpline(Cubic(Line())), OnGrid())
# sitpc = scale(itpc, ...)

In [ ]:
tmp =RectangleGrid(pgrid, vgrid, vbgrid)

tmp

In [ ]:
using GridInterpolations

In [ ]:
[pgrid, vgrid, vbgrid]

In [ ]:
sitpc(58.80649063906391, )

In [ ]:
V0 = 100
p = 58.80649063906391
vb = 64.96307165308276
v = log(100/vb)

sitpc[p, v, vb]

In [ ]:
sitpc = scale(itpc, pgrid, vgrid, vbgrid)

In [ ]:
mgrid = [(p, v, vb) for p=pgrid, v=vgrid, vb=vbgrid]

In [ ]:
grid = RectangleGrid([0., 0.5, 1.],[0., 0.5, 1.])

In [ ]:
using JLD

In [ ]:
save("data.jld", "data", cube)


In [ ]:
cube2 = load("data.jld")["data"]

In [ ]:
using Interpolations
# Maybe pass a single vmax
tic()
vbgrid = linspace(50, 70, 5)
debtgrid = @spawn [fun_wrapper(100, vb, pardict) for vb=vbgrid]
debtvec = fetch(debtgrid)
itp = interpolate(debtvec, BSpline(Cubic(Line())), OnGrid())
sitp = Interpolations.scale(itp, vbgrid) # Scale
vbgrid_refined_size = 10^4
vbgrid_refined = linspace(vbgrid[1], vbgrid[end], vbgrid_refined_size)
debtref = [sitp[x] for x in vbgrid_refined]
toc()

In [ ]:
using Plots
plot(vbgrid_refined, debtref - pardict["p"] * pardict["m"]) 

In [ ]:
tmp = debtref - pardict["p"] * pardict["m"]
# minimum(abs.(tmp))
vbgrid_refined[findmin(abs.(tmp))[2]]

In [ ]:
vbgrid_refined[findmin(abs.(tmp))[2]]

In [ ]:
minimum(abs.(tmp))

In [ ]:
?afun.get_debt_price()

In [ ]:
fun_wrapper(pardict)

In [ ]:
fun_wrapper(100, 1, pardict)

In [ ]:
fun_wrapper(100, .5, pardict)

In [ ]:
fun_wrapper(100, 1, pardict)

In [ ]:
fun_wrapper(100, .5, pardict)

In [ ]:
pardict["p"]=58.806491
pardict

In [ ]:
fun_wrapper(100, 1, pardict)

In [ ]:
?afun.zhi_bond_price()

In [ ]:
function debt_pr_wrapper(vt, vmax, vb, vbh, ttm_grid_size, dict)
   return afun.get_debt_price(vt, vb, vbh, ttm_grid_size,
                            dict["m"], dict["c"], dict["p"], 
                            dict["r"], dict["gross_delta"], dict["iota"], 
                            dict["xi"], dict["kappa"], dict["alpha"], dict["pi"],
                            dict["lambda"], dict["sigmal"], dict["sigmah"]) 
    
end

In [ ]:
tic()
a = debt_pr_wrapper(.5, 1.2, 60, 62, 25, pardict)
toc()
a

In [ ]:
tic()
a = debt_pr_wrapper(.5, 1.2, 60, 62, 25, pardict)
toc()
a

In [ ]:
?Interpolations.scale

In [ ]:
print(string("shgjrhgfkj: " , 3.545))

In [ ]:
function bond_vmax_wrapper(V0, dict)
    return afun.get_bond_vmax(V0, dict["m"], dict["c"], dict["p"], 
                         dict["sigmah"], dict["r"], 
                         dict["gross_delta"], dict["iota"], 
                         dict["xi"], dict["kappa"],
                         dict["alpha"], dict["pi"],)
end

In [ ]:
bond_vmax_wrapper(100, pardict)

In [ ]:
?afun.get_bond_vmax

In [ ]:
println(string("Bond Percentage Difference: ", per_diff))

In [ ]:
function tmp(a, vmax=NaN)
    if !isnan(vmax)
        println("Returning vmax")
        return a, vmax
    else
        return a
    end
end

In [ ]:
tmp(3)

In [ ]:
a, vmax= tmp(4, 6)

In [ ]:
a

In [ ]:
vmax

In [ ]:
_, vmax = tmp(5, 7)

In [ ]:
vmax

In [ ]:
function vol_shock_cf_integrand_wrapper(vt, v, u, ttm, vb, vbh, dict)
    return afun.vol_shock_cf_integrand(vt, v,
                                      v - log(vbh / vb),
                                      vb, vbh, 
                                      u, ttm,
                                      dict["c"], dict["p"],
                                      dict["sigmal"], dict["sigmah"], 
                                      dict["r"], dict["gross_delta"],
                                      dict["xi"], dict["kappa"],
                                      dict["alpha"], dict["pi"],
                                      dict["lambda"]) * vb * exp(v) * (v > 0)
end

function approx_bond_int2(vt, v_upper_thresh, ttm, rdisc, rfbond_ttm, dict)
    return dict["lambda"] * exp(-(rdisc + dict["lambda"]) * ttm) * 
           rfbond_ttm * afun.psi_v_td(vt, v_upper_thresh, ttm, dict["sigmal"], 
                                              dict["r"], dict["gross_delta"])
end

In [ ]:
# function bond_pr_vol_cf(v, ttm, vb, vbh, dict)
#     tmp = @spawn [ vol_shock_cf_integrand_wrapper(vt, v, u, ttm, dict)*(1/1000)^2 for 
#                     u=linspace(0.0001, ttm, 1000), vt=linspace(0.0, 1, 1000)]

#     return  sum(fetch(tmp))
# end

In [ ]:
approx_bond_int2(.5, 1.2, .7, 
                 afun.rdisc(pardict["r"], pardict["xi"], pardict["kappa"]),
                 afun.rfbond_price(.7, pardict["c"], pardict["p"], pardict["r"], 
                                        pardict["xi"], pardict["kappa"]),
                 pardict)

In [ ]:
function bond_pr_vol_cf2(vt, ttm, vmax, vb, vbh, dict)
    tauN = 10^3
    vN = 10^3
    dt1 = (ttm - 1e-4)/tauN
    dv = vmax/vN
    cf1_mat = @spawn [vol_shock_cf_integrand_wrapper(vt, v, u, ttm, vb, vbh, dict) for 
                    u=linspace(1e-4, ttm, tauN), v=linspace(0.0, vmax, vN)]
    cf1 = sum(fetch(cf1_mat)) * dt1 * dv
    
    N = 10^4
    dt2 = ttm/N
    rdisc = afun.rdisc(dict["r"], dict["xi"], dict["kappa"])
    # Define auxiliary function:
    get_rfbond(ttm) = afun.rfbond_price(ttm, dict["c"], dict["p"], dict["r"], 
                                        dict["xi"], dict["kappa"])
    cf2_vec = @spawn [approx_bond_int2(vt, vmax, tau, rdisc, get_rfbond(tau), dict) for 
            tau=linspace(dt2, ttm, N)]
    cf2 = sum(fetch(cf2_vec)) * dt2
#     cf2 = sum(cf2_vec) * dt2
#     return sum(fetch(cf1_mat)) * dt1 * dv + cf2 
#     return sum(fetch(cf1_mat)) * dt1 * dv + sum(fetch(cf2_vec)) * dt2
    return cf1 + cf2
end

In [ ]:
tic()
a = bond_pr_vol_cf2(.5, .7, 1.2, 60, 62, pardict)
toc()
a

In [ ]:
sum(fetch(a))

In [ ]:
function bond_pr_main(vt, ttm, vmax, vb, vbh, dict)
    
    if vt <= 0
        return afun.on_default_payoff(vt, vb, ttm,
                                      dict["m"], dict["c"], dict["p"],
                                      dict["r"], dict["xi"], dict["kappa"], dict["alpha"])
    else
        # function to get vmax
        # function to get rfbond
        rfbond = afun.rfbond_price(ttm, dict["c"], dict["p"], dict["r"], 
                                        dict["xi"], dict["kappa"])
        
        if vt > vmax
            return rfbond
        else
            cf0 = afun.no_vol_shock_cf_pv(vt, vb, ttm,
                         dict["m"], dict["c"], dict["p"],
                         dict["sigmal"], dict["r"],
                         dict["gross_delta"],
                         dict["xi"], dict["kappa"], 
                         dict["alpha"], dict["lambda"])
            
            cf1 = bond_pr_vol_cf2(vt, ttm, vmax, vb, vbh, dict)
            
            price = min(cf0 + cf1, rfbond)
            
            return price
        end
    end
end

In [ ]:
tic()
a = bond_pr_main(.5, 1, 1.2, 60, 62, pardict)
toc()
a

In [ ]:
function bond_pr_wrapper(vt, ttm, vmax, vb, vbh, dict)
   return afun.bond_pr_main(vt, ttm, vmax, vb, vbh,
                            dict["m"], dict["c"], dict["p"], 
                            dict["r"], dict["gross_delta"], 
                            dict["xi"], dict["kappa"], dict["alpha"], dict["pi"],
                            dict["lambda"], dict["sigmal"], dict["sigmah"]) 
    
end

In [ ]:
tic()
a = bond_pr_wrapper(.5, .11, 1.2, 60, 62, pardict)
toc()
a

In [ ]:
tic()
tmp = @spawn [bond_pr_wrapper(v, 1, 1.2, 60, 62, pardict) for v=linspace(.0, .5,100)]
toc()

In [ ]:
tic()
bpvec = fetch(tmp)
toc()

In [ ]:
tic()
tmp = @spawn [bond_pr_main(v, 1, 1.2, 60, 62, pardict) for v=linspace(.0, .5,100)]
toc()

In [ ]:
tic()
bpvec = fetch(tmp)
toc()

In [ ]:
# Missing:
# function to get vmax
# interpolate bond_pr(tau)
# compute debt price

In [ ]:
tic()
ttm_grid = linspace(.001, 1,20)
tmp = @spawn [bond_pr_wrapper(.5, ttm, 1.2, 60, 62, pardict) for ttm=ttm_grid]
ttmvec = fetch(tmp)
toc()

In [ ]:
using Interpolations

In [ ]:
itp = interpolate(ttmvec, BSpline(Cubic(Line())), OnGrid())
sitp = scale(itp, ttm_grid)

In [ ]:
# Get Debt Price

tic()

m = pardict["m"]
t0 = .0001

# Grid for Interpolation
N1 = 25
ttm_grid = linspace(t0, m, N1)

# Refined Grid
N2 = 10^3
ttm_grid_refined = linspace(t0, m, N2)
dt = (m - t0)/N2

tmp = @spawn [bond_pr_wrapper(.5, ttm, 1.2, 60, 62, pardict) for ttm=ttm_grid]
ttmvec = fetch(tmp)

# Interpolate
itp = interpolate(ttmvec, BSpline(Cubic(Line())), OnGrid())
sitp = scale(itp, ttm_grid) # Scale

# Integrate
tbp = (1/m) * sum([sitp[x] for x=ttm_grid_refined]) * dt

toc()

tbp

In [ ]:
using Plots

In [ ]:
plot(linspace(.001, 1., 1000), tbp)

In [ ]:
ttm_grid

In [ ]:
ttm_grid = linspace(.001, 1,20)
A = [ttm_grid, ttmvec]

In [ ]:
size(A)

In [ ]:
A_x = 1.:2.:40.
B = [log(x) for x in A_x]

B

In [ ]:
tic()
ttmvec = fetch(tmp)
toc()

In [ ]:
plot(ttmvec)

In [ ]:
ttmvec

In [ ]:
using Plots

In [ ]:
plot(bpvec)

In [ ]:
tic()
b = @spawn [ vol_shock_cf_integrand_wrapper(.5, .7, u, 1, pardict)*(1/1000)^2 for 
            u=linspace(0.0001, 1, 1000), vt=linspace(0.0, 1, 1000)]

toc()
a = sum(fetch(b))

In [ ]:
tic()
a = bond_pr_vol_cf(.5, .7,pardict)
toc()

In [ ]:
sum(fetch(a))

In [ ]:
sum(fetch(a))

In [ ]:
Pkg.add("Cubature")

In [ ]:
tic()
b = @spawn [ vol_shock_cf_integrand_wrapper(vt, .5, u, .7, pardict)*(1/1000)^2 for 
            u=linspace(0.0001, .7, 1000), vt=linspace(0.0, 1, 1000)]

toc()
a = sum(fetch(b))

In [ ]:
fetch(a)

In [ ]:
linspace(0, 1, 10)

In [ ]:
struct Foo
           bar
           baz::Int
           qux::Float64
       end

In [ ]:
mutable struct ArturRMModel
        
        alpha::Float64
        pi::Float64
        r::Float64
        gross_delta::Float64

        vb::Float64
        vbh::Float64

        xi::Float64
        kappa::Float64

        lambda::Float64
        sigmal::Float64
        sigmah::Float64

        m::Float64
        c::Float64
        p::Float64

        new(alpha)
end

In [ ]:
arm = ArturRMModel(ones(14,1))

In [ ]:
arm.m 

In [ ]:
Foo(baz) = 3

In [ ]:
fieldnames(Foo)

In [ ]:
u = Foo(3, 4, 5)

In [ ]:
u.qux

In [ ]:
u = Tuple{Float64,Float64}[]

In [ ]:
u[1] = 3.

In [ ]:
log(100 / pardict["vb"])

In [ ]:
analytic_functions.vol_shock_cf_integrand(log(110/60), log(100/60), log(110/62), 60, 62, .5, .7, 3, 60,
                                   .2, .3, .08, .02, 1, .004, .6, .27, .3)

In [ ]:
analytic_functions.no_vol_shock_cf_pv(log(100/60), 60, .7, 1, 3, 60, .25, .08, .02, 1, .004, .6, .3)

In [ ]:
analytic_functions.default_payoff_pv_fun(log(100/60), 60, .7, 1., .25, .08, .02, 1, .004, .6, .3)

In [ ]:
analytic_functions.maturity_payoff_pv_fun(log(100/60), .7, 60, .25, .08, .02, 1, .004,  .3)

In [ ]:
analytic_functions.coupon_pv_fun(log(100/60), .7, 3, .2, .08, .02, 1, .004, .3)

In [ ]:
(.6 * 60 * exp(log(100/60)) / 1)

In [ ]:
analytic_functions.rf_debt(1, 3, 65, .08, 1, .004)

In [ ]:
analytic_functions.on_default_payoff(log(120/60), 60, .7, 1, 3, 65, .08, 1, .004, .6)

In [ ]:
(.6 * 60 * exp(log(100/60)) / 1) * (30 > (.6 * 60 * exp(log(100/60)) / 1))

In [ ]:
(3 / analytic_functions.rdisc(.08, 1, .004) + exp(-analytic_functions.rdisc(.08, 1, .004) * .7)) * (65 - 3 / analytic_functions.rdisc(.08, 1, .004))

In [ ]:
exp(-analytic_functions.rdisc(.08, 1, .004) * .7)

In [ ]:
(3 / analytic_functions.rdisc(.08, 1, .004) + exp(-analytic_functions.rdisc(.08, 1, .004) * .7))

In [ ]:
(65 - 3 / analytic_functions.rdisc(.08, 1, .004))

In [ ]:
analytic_functions.rf_debt(1, 3, 65, .08, 1, .004)

In [ ]:
analytic_functions.dv_psi_v_td(log(120/60), log(100/60), .7, .2, .08, .02)

In [ ]:
using Distributions
pdf(Normal(), 0.)

In [ ]:
analytic_functions.psi_v_td(log(120/60), log(100/60), .7, .2, .08, .02)

In [ ]:
analytic_functions.G_pvs(log(100/60), .2, .2, .08, .02, 1, .004, .3)

In [ ]:
analytic_functions.q1_pvs(log(100/60), .2, .2, .08, .02, 1, .004, .3)

In [ ]:
analytic_functions.q2_pvs(log(100/60), .2, .2, .08, .02, 1, .004, .3)

In [ ]:
analytic_functions.zhat_pvs(.2, .08, .02, 1, .004, .3)

In [ ]:
af.rdisc_pvs(.08, 1, .004, .3)

In [ ]:
analytic_functions.zhi_vb(1, 3, 60, .2, .08, .02, 0.0, 1, .004, .6, .27)

In [ ]:
analytic_functions.bsm_K(log(100/60), .2, .3, .7, .2, .9)

In [ ]:
analytic_functions.bsm_gamma(.2, .08, .02)

In [ ]:
analytic_functions.bsm_A(log(100/60), .8, .25, .08, .02, .9)

In [ ]:
analytic_functions.zhi_eq(log(100/60), 1, 3, 60, .2, .08, .02, .0, 1, .004, .6, .27)

In [ ]:
max(20, 1e-4)

In [ ]:

bsm_numerator2(1, 3, 60, .2, .08, .02, 1, .004)

In [ ]:
bsm_denominator(1, .2, .08, .02, 0.0, 1, .004, .6)

In [ ]:
bsm_numerator1(1, 3, 60, .2, .08, .02, 1, .004, .27)

In [ ]:
auxiliary_functions.bsm_b(x, m, sigma, r, gross_delta, xi, k)

In [ ]:
auxiliary_functions.bsm_B(log(100/60), 1, .2, .08, .02)

In [ ]:
auxiliary_functions.bsm_b(log(100/60), 1, .2, .08, .02, 1, .004)

In [ ]:

zhi_bond_price(60, log(100/60), 1, 1, 3, 69, .2, .08, .02, 1, .004, .6, .27)

In [ ]:
rdisc(.08, 1, .002)

In [ ]:
bsm_F(.5, .7, .2, .08, .02) #, 1, .004)

In [ ]:
bsm_G(.5, .7, .2, .08, .02, 1, .004)

In [ ]:
cdf(Normal(), bsm_h1(.5, .7, .2, .08, .02))

In [ ]:
bsm_eta(.5, .08, .02)

In [ ]:
using Distributions

In [ ]:
bsm_z(.25, .08, .02)

In [ ]:
quantile.(Normal(), [0.5, 0.975])

In [ ]:
cdf(Normal(), 1.96)

In [ ]:
log(100/60)

In [ ]:
MethodError: objects of type Symbol are not callable

In [ ]:
function fff(x, a=1)
    return exp(cos(a * x))
end


In [ ]:
fff(pi,2)